In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os

In [4]:
import torch
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import random_split, Subset
import pytorch_lightning as pl
from tqdm.notebook import tqdm, trange
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from IPython.display import clear_output
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

In [5]:
from system import System
from conflab.data_loaders.pose import ConflabPoseExtractor, ConflabToKinetics
from conflab.data_loaders.accel import ConflabAccelExtractor
from conflab.data_loaders.person import ConflabMultimodalDataset, ConflabLabelExtractor
from conflab.constants import conflab_pose_path, midge_data_path, conflab_speaking_status_path
from conflab.constants import vid2_start, vid3_start

In [6]:
from conflab.data_loaders.utils import time_to_seg
from datetime import timedelta

In [7]:
vid2_seg8_start = vid2_start + timedelta(minutes=14)
vid2_len = (vid3_start - vid2_seg8_start).total_seconds()

In [8]:
vid2_len

217.0

In [9]:
def do_fold(train_ds, test_ds, model_name='resnet'):
    # data loaders
    data_loader_train = torch.utils.data.DataLoader(
        train_ds, batch_size=100, shuffle=True, num_workers=10,
        collate_fn=None)
    data_loader_val = torch.utils.data.DataLoader(
        test_ds, batch_size=100, shuffle=False, num_workers=10,
        collate_fn=None)

    system = System(model_name)
    trainer = pl.Trainer(
        callbacks=[EarlyStopping(monitor="val_loss", mode="min")],
        accelerator='gpu',
        log_every_n_steps=1,
        max_epochs=-1)
    trainer.fit(system, data_loader_train, data_loader_val)

    trainer.test(system, data_loader_val)
    return system.test_results

In [10]:
def get_metrics(outputs, labels, type='binary'):
    if type == 'binary':
        proba = torch.sigmoid(outputs)
        pred = (proba > 0.5)

        correct = pred.eq(outputs.bool()).sum().item()
        return {
            'auc': roc_auc_score(labels, proba),
            'correct': correct
        }
    elif type == 'regression':
        return {
            'mse': torch.nn.functional.mse_loss(outputs, labels, reduction='mean'),
            'l1': torch.nn.functional.l1_loss(outputs, labels, reduction='mean')
        }

In [11]:
def do_run(dataset, model_name, metrics_name='binary'):
    seed = 22
    cv_splits = KFold(n_splits=10, random_state=seed, shuffle=True).split(range(len(ds)))

    outputs = torch.empty((len(ds),))
    for f, (train_idx, test_idx) in enumerate(cv_splits):
        # create datasets    
        train_ds = Subset(dataset, train_idx)
        test_ds = Subset(dataset, test_idx)

        fold_outputs = do_fold(train_ds, test_ds, model_name)
        outputs[test_idx] = fold_outputs['proba'].cpu()
        clear_output(wait=True)

    labels = torch.Tensor(ds.get_all_labels())
    run_metrics = get_metrics(outputs, labels, metrics_name)

    return outputs, run_metrics

In [12]:
pose_extractor = ConflabPoseExtractor(conflab_pose_path)
pose_extractor.load_from_pickle('./tracks.pkl')

In [13]:
accel_extractor = ConflabAccelExtractor(midge_data_path)
label_extractor = ConflabLabelExtractor(os.path.join(conflab_speaking_status_path, 'speaking'))

In [14]:
label_extractor.labels[0].keys()

Index(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2',
       '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31',
       '32', '33', '34', '35', '3', '4', '5', '6', '7', '8', '9'],
      dtype='object')

In [15]:
# make windowed examples using the pose tracks.
examples = pose_extractor.make_examples()
# compose the dataset
ds = ConflabMultimodalDataset(examples, {
    'accel': accel_extractor,
    'label': label_extractor
})

100%|██████████| 8/8 [00:00<00:00, 560.45it/s]


In [16]:
for i in trange(len(ds)):
    batch = ds[i]

  0%|          | 0/43440 [00:00<?, ?it/s]

In [17]:
outputs, metrics = do_run(ds, 'alexnet', 'binary')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | MyAlexNet | 180 K 
------------------------------------
180 K     Trainable params
0         Non-trainable params
180 K     Total params
0.724     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x1088 and 384x192)

In [ ]:
metrics

{'auc': 0.8161846056582899, 'correct': 308}

In [ ]:
outputs, metrics = do_run(ds, 'alexnet', 'binary')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | MyAlexNet | 180 K 
------------------------------------
180 K     Trainable params
0         Non-trainable params
180 K     Total params
0.724     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_auc            0.8045634920634921
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
metrics

{'auc': 0.8096451714872768, 'correct': 326}